# [Data Train] Competição ML #2 - SECITEC 2020 IFG
## CRISP-DM: Data Preparation
**Autor:** Wanderson Marques - wdsmarques@gmail.com

Esse Jupyter Notebook contém o **pré-processamento** do conjunto de dados. Considerando a metodologia CRISP-DM, essa atividade refere-se à terceira fase, a preparação dos dados. 

<img src="imgs/dataPreparation.jpg" />

### Carregar bibliotecas

In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split

### Carregar dataset

In [2]:
dataset = pd.read_csv('datasets/treino.csv')
dataset.head()

,id,idademae,estadocivilmae,catprenatal,qtdsemanas,tipoparto,peso,malformacao,sexo,apgar1,apgar5,obito
0,2708,13.0,solteira,de4a6,36.0,vaginal,2440.0,nao,f,9.0,10.0,0
1,14097,25.0,casada,7mais,39.0,cesareo,2850.0,nao,f,9.0,10.0,0
2,5360,27.0,solteira,7mais,38.0,cesareo,3350.0,nao,f,9.0,10.0,0
3,15902,28.0,solteira,7mais,41.0,vaginal,3065.0,nao,m,8.0,9.0,0
4,8985,21.0,solteira,de4a6,39.0,cesareo,3085.0,nao,f,8.0,9.0,0


In [3]:
dataset.shape

(11389, 12)

### Descartar o identificador

In [4]:
dataset.drop(['id'], axis=1, inplace=True)
dataset.head()

,idademae,estadocivilmae,catprenatal,qtdsemanas,tipoparto,peso,malformacao,sexo,apgar1,apgar5,obito
0,13.0,solteira,de4a6,36.0,vaginal,2440.0,nao,f,9.0,10.0,0
1,25.0,casada,7mais,39.0,cesareo,2850.0,nao,f,9.0,10.0,0
2,27.0,solteira,7mais,38.0,cesareo,3350.0,nao,f,9.0,10.0,0
3,28.0,solteira,7mais,41.0,vaginal,3065.0,nao,m,8.0,9.0,0
4,21.0,solteira,de4a6,39.0,cesareo,3085.0,nao,f,8.0,9.0,0


### Separar conjuntos de treino (70%) e validação (30%)

O conjunto de teste será utilizado posteriormente, para verificar a capacidade preditiva do modelo. Ele é isolado para que não receba nenhum tipo de viés.

In [5]:
train, teste_split = train_test_split(dataset.copy(), test_size=0.3)

### Gravar conjuntos de validação

In [6]:
teste_split.to_csv('datasets/teste_split.csv', index=False)

### Tratar valores nulos

São tratamentos comuns para valores nulos:
- Exclusão do atributo (caso ele seja nulo para grande parte das instâncias)
- Exclusão da instância (caso ela seja nula para grande parte dos atributos)
- Imputação por estatísticas simples, como média, mediana ou moda (podem ser calculadas para sub-amostras)
- Imputação por regressão e modelos preditivos

In [7]:
train.isnull().sum()

idademae            0
estadocivilmae     69
catprenatal        32
qtdsemanas        157
tipoparto           4
peso                1
malformacao       207
sexo               32
apgar1             75
apgar5             74
obito               0
dtype: int64

In [8]:
# Imputar a moda em todas as variáveis
for col in train.columns:
    train.loc[train[col].isnull(), col] = (train.mode())[col][0]

In [9]:
train.isnull().sum()

idademae          0
estadocivilmae    0
catprenatal       0
qtdsemanas        0
tipoparto         0
peso              0
malformacao       0
sexo              0
apgar1            0
apgar5            0
obito             0
dtype: int64

### Tratar outliers

Não será realizado nenhum tratamento

### Converter variáveis categóricas em dummies

Cada valor possível para uma variável categórica devem ser transformado em um atributo binário (0 ou 1)

In [10]:
train = pd.get_dummies(train)

In [11]:
# Evitar dummy trap excluindo valores de referência
train.drop(['estadocivilmae_solteira', 'catprenatal_7mais', 'tipoparto_cesareo', 'malformacao_nao', 'sexo_m'], axis=1, inplace=True)
train.head()

,idademae,qtdsemanas,peso,apgar1,apgar5,obito,estadocivilmae_casada,estadocivilmae_separada,estadocivilmae_uniao,estadocivilmae_viuva,catprenatal_de1a3,catprenatal_de4a6,catprenatal_nenhuma,tipoparto_vaginal,malformacao_sim,sexo_f
8065,28.0,38.0,3970.0,9.0,10.0,0,1,0,0,0,0,0,0,0,0,0
6134,20.0,39.0,3420.0,9.0,10.0,0,0,0,0,0,0,0,0,0,0,0
3662,31.0,39.0,2740.0,9.0,10.0,0,0,0,0,0,1,0,0,0,0,0
3635,18.0,23.0,535.0,1.0,2.0,1,1,0,0,0,0,1,0,1,0,1
5342,22.0,38.0,3530.0,8.0,9.0,0,0,0,1,0,0,0,0,1,0,0


### Gravar conjuntos de treino pré-processado

Após a finalização do pré-processamento, os dados transformados são salvos.

In [12]:
train.to_csv('datasets/train-preprocessado.csv', index=False)